In [1]:
!nvidia-smi

Sat Feb 20 10:15:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#download Food-101 data
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!gunzip /content/food-101.tar.gz
!tar -xvf /content/food-101.tar

In [3]:
#get classes, and file names of train and validation set  
with open('/content/food-101/meta/classes.txt', 'r') as f:
    classes = f.readlines()
classes = [c.replace('\n', '') for c in classes]
print(f'num classes {len(classes)}')
with open('/content/food-101/meta/train.txt', 'r') as f:
    train = f.readlines()
train = [t.replace('\n', '') for t in train]
with open('/content/food-101/meta/test.txt', 'r') as f:
    val = f.readlines()
val = [v.replace('\n', '') for v in val]

#prepare train and validation data
import os
from tqdm.notebook import tqdm
from PIL import Image

data_dir = '/content/data'
for f in ['train', 'val']:
    for c in classes:
        os.makedirs(f'{data_dir}/{f}/{c}', exist_ok=True)

base_dir = '/content/food-101/images'
print('preparing train data')
#resize images to 224 by 224
for image in tqdm(train):
    path_in = f'{base_dir}/{image}.jpg'
    I = Image.open(path_in).resize((224, 224))
    path_out = f'{data_dir}/train/{image}.jpg'
    I.save(path_out)
print('preparing validation data')
for image in tqdm(val):
    path_in = f'{base_dir}/{image}.jpg'
    I = Image.open(path_in).resize((224, 224))
    path_out = f'{data_dir}/val/{image}.jpg'
    I.save(path_out)

num classes 101
preparing train data



preparing validation data


In [4]:
#calculate the means and standard deviations of train data
import cv2
import numpy as np
from pathlib import Path

path_images = Path('/content/data/train')
num_images = len(list(path_images.glob('**/*.jpg')))
num_channels = 3
num_pixels = 0
channel_sum = np.zeros(num_channels)
channel_square_sum = np.zeros(num_channels)

for image in tqdm(path_images.glob('**/*.jpg'), total=num_images):
    img = cv2.imread(str(image))
    img = img / 255.0
    num_pixels += img.size / num_channels
    channel_sum += np.sum(img, axis=(0, 1))
    channel_square_sum += np.sum(np.square(img), axis=(0, 1))
mean = list(channel_sum / num_pixels)
std = list(np.sqrt(channel_square_sum / num_pixels - np.square(mean)))

mean[::-1], std[::-1]

([0.5447903194476672, 0.44346403124059985, 0.3440857279029884],
 [0.2707449881924057, 0.2728363337042646, 0.2766531068472587])

In [5]:
#install timm and clone T2T-ViT repository
!pip install timm
!git clone https://github.com/yitu-opensource/T2T-ViT

     |████████████████████████████████| 245kB 8.6MB/s 
Cloning into 'T2T-ViT'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 87 (delta 48), reused 13 (delta 7), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [6]:
#train T2T-ViT on Food-101
#I modified the line 43 in the code of token_performer.py as follows
#before : return torch.exp(wtx - xd) / math.sqrt(self.m)
#after : return torch.exp((wtx - xd) - torch.max((wtx - xd), dim=-1, keepdim=True).values + self.epsilon) / math.sqrt(self.m)
import os
os.chdir('/content/T2T-ViT')

PATH_TO_DATA = '/content/data'

!bash distributed_train.sh 1 "$PATH_TO_DATA"\
    --model T2t_vit_14\
    --batch-size 32\
    --num-classes 101\
    --img-size 224\
    --mean 0.545 0.443 0.344\
    --std 0.271 0.273 0.277\
    --lr 1e-4\
    --epochs 10\

Training with a single process on 1 GPUs.
adopt performer encoder for tokens-to-token
Model T2t_vit_14 created, param count: 21199435
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (0.545, 0.443, 0.344)
	std: (0.271, 0.273, 0.277)
	crop_pct: 0.9
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 20
Train: 0 [   0/2367 (  0%)]  Loss:  4.625757 (4.6258)  Time: 2.075s,   15.42/s  (2.075s,   15.42/s)  LR: 1.000e-06  Data: 1.217 (1.217)
Train: 0 [  50/2367 (  2%)]  Loss:  4.743123 (4.6779)  Time: 0.278s,  115.12/s  (0.315s,  101.46/s)  LR: 1.000e-06  Data: 0.004 (0.028)
Train: 0 [ 100/2367 (  4%)]  Loss:  4.647557 (4.6723)  Time: 0.279s,  114.77/s  (0.298s,  107.39/s)  LR: 1.000e-06  Data: 0.004 (0.016)
Train: 0 [ 150/2367 (  6%)]  Loss:  4.670959 (4.6717)  Time: 0.279s,  114.78/s  (0.293s,  109.25/s)  LR: 1.000e-06  Data: 0.004 (0.012)
Train: 0 [ 200/2367 (  8%)]  Loss:  4.595654 (4.6675)  Time: 0.280